In [26]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('gurgaon_properties_post_feature_selection.csv')

In [30]:
df.head()

,sector,property_type,bedRoom,bathroom,balcony,floor_category,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,price
0,108.0,0.0,2,2,2.0,0.0,1.0,695.0,0,0,1,1.0,0.45
1,89.0,0.0,3,3,2.0,2.0,0.0,2045.0,0,0,2,1.0,1.20
2,2.0,0.0,3,3,4.0,2.0,0.0,1900.0,1,0,1,1.0,0.91
3,11.0,0.0,3,4,2.0,0.0,1.0,2525.0,0,0,1,2.0,3.35
4,11.0,0.0,2,2,2.0,2.0,3.0,1250.0,0,0,1,2.0,1.90


In [32]:
# one hot encode -> sector, balcony, agePossession, furnishing type, luxury category, floor category

In [34]:
X = df.drop(columns=['price'])
y = df['price']

In [145]:
from sklearn.model_selection import  KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier, XGBRegressor

property_type
0.0    2804
1.0     750
Name: count, dtype: int64

In [157]:
columns_to_encode = ['sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [159]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

In [217]:
# Creating a column transformer for preprocessing

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), columns_to_encode)
    ], 
    remainder='passthrough',force_int_remainder_cols=False
)

In [219]:
# creating pipeline
pipeline = Pipeline([
    ("preprocessor",preprocessor),
    ("regressor",XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    random_state=42)
)
     ])

In [221]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [222]:
scores.mean()

0.8962195735102334

In [223]:
scores.std()

0.00998107604931252

In [224]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [225]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['sector', 'balcony',
                                                   'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_cat...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [226]:
y_pred = pipeline.predict(X_test)

In [227]:
y_pred = np.expm1(y_pred)

In [228]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(np.expm1(y_test),y_pred)

0.5414058186233295